In [1]:
import sys
if (path := "C:/Users/Tom/PycharmProjects/python-hvac") not in sys.path:
    sys.path.append(path)

In [2]:
from hvac import Quantity, print_doc_string

# Chapter 4: Internal Heat Gains in a Thermal Zone
---

Equipment, lighting, and people in a thermal zone of the building give off heat to the space. Data to estimate the internal heat gains from equipment, lighting and people can be found e.g. in ASHRAE Fundamentals 2017, Chapter 18 (Nonresidential Cooling and Heating Load Calculations). Three classes are available to represent the different types of internal heat gain:
- class `EquipmentHeatGain`
- class `LightingHeatGain`
- class `PeopleHeatGain`

In [3]:
from hvac.cooling_load_calc import (
    EquipmentHeatGain,
    LightingHeatGain,
    PeopleHeatGain
)

Q_ = Quantity

## Internal Heat Gain from Equipment

Module `equipment.py` in subpackage `cooling_load_calc.internal_heat_gains` defines a number of equipment classes:
- class `Machine`
- class `HoodedCookingAppliance`
- class `OfficeAppliance`
- class `OfficeEquipment`
- class `GenericAppliance`

In [4]:
from hvac.cooling_load_calc import (
    Machine,
    HoodedCookingAppliance,
    OfficeAppliance,
    OfficeEquipment,
    GenericAppliance
)

Objects from these classes are instantiated by calling the class method `create(...)`.

### Machine

In [5]:
print_doc_string(Machine.create)

Creates a `Machine` object.

Parameters
----------
name:
    Identifies the machine.
P_motor:
    Motor power nameplate rating.
eta_motor:
    Motor efficiency. See e.g. ASHRAE Fundamentals 2017, Chapter 18,
    Tables 4A and 4B.
configuration:
    See `Enum` subclass `Configuration` in class `Machine`:
    - `Configuration.MOTOR_AND_MACHINE`: the motor and the driven
       machine are both in the conditioned space.
    - `Configuration.ONLY_MACHINE`: only the driven machine is in the
       conditioned space.
    - `Configuration.ONLY_MOTOR`: only the motor is in the conditioned
       space (e.g. a fan in the conditioned space that exhausts air
       outside that space).
schedule:
    Function with signature `f(t_sol_sec: float) -> float` that takes the
    solar time `t_sol_sec` in seconds since midnight (0 s) and returns
    a float between 0 and 1, where 0 stands for completely off and 1
    for running at full power.
F_rad:
    The radiative fraction is the radiative part of th

A **schedule** allows to change the operation of a machine in the course of the day. It must be a function that takes the time of the day and returns a fraction of the rated motor load. This fraction is the product of the motor use factor and the motor load factor (see ASHRAE Fundamentals 2017, Chapter 18, §2.3).

A schedule expects the time of the day to be expressed in solar seconds since midnight. However, we are used to clock time. Therefore, the package `cooling_load_calc` has functions to convert solar seconds since midnight to clock time and vice versa.

In [6]:
from hvac.sun import ReferenceDates
from hvac.cooling_load_calc import convert_to_clock_time

In [7]:
print_doc_string(convert_to_clock_time)

Converts local solar time in seconds from midnight to local solar time in
date-time format and to local standard time in date-time format.

Parameters
----------
t_sol_sec:
    Solar time in seconds from midnight (midnight is 0 s).
date:
    The date of the day under consideration.
L_loc:
    The longitude of the location under consideration. East from Greenwich
    is a positive angle, and west is a negative angle.
tz_loc:
    The time zone of the location according to the tz database notation.

Returns
-------
A 2-tuple with:
-   Python datetime object representing the date and local standard time that
    corresponds with the given solar time in seconds.
-   Python datetime object representing the date and local solar time that
    corresponds with the given solar time in seconds.

Parameter Types & Defaults
--------------------------
t_sol_sec: float
date: datetime.date
L_loc: pint.Quantity
tz_loc: str


In [8]:
L_loc = Q_(3.8, 'deg')
date = ReferenceDates.get_date_for('Jun')
tz_loc = "Europe/Brussels"

def eqp_schedule(t_sol_sec: float) -> float:
    t_clock, _ = convert_to_clock_time(t_sol_sec, date, L_loc, tz_loc)
    t_hour = t_clock.time().hour
    if 8 <= t_hour < 12:
        return 1.0
    elif 12 <= t_hour < 14:
        return 0.5
    elif 14 <= t_hour < 17:
        return 0.8
    else:
        return 0.0


In [9]:
machine = Machine.create(
    name='machine_1',
    P_motor=Q_(7.5, 'kW'),
    eta_motor=Q_(91.7, 'pct'),
    configuration=Machine.Configuration.MOTOR_AND_MACHINE,
    schedule=eqp_schedule
)

### Hooded Cooking Appliance

In [10]:
print_doc_string(HoodedCookingAppliance.create)

Creates a `HoodedCookingAppliance` object.
(See ASHRAE Fundamentals 2017, Chapter 18, Tables 5A to 5E).

Parameters
----------
name:
    Identifies the appliance.
P_rated:
    Nameplate or rated energy input rating.
F_U:
    Usage factor applied to the nameplate rating that determines the
    average rate of appliance energy consumption.
F_rad:
    The radiative fraction is the radiative part of the cooking
    appliance heat gain that goes to the space.
schedule:
    Function with signature `f(t_sol_sec: float) -> float` that takes the
    solar time `t_sol_sec` in seconds from midnight (0 s) and returns
    a float between 0 and 1, where 0 stands for completely off and 1
    for running at full power.

Parameter Types & Defaults
--------------------------
name: 'str'
P_rated: 'Quantity'
F_U: 'float'
F_rad: 'float'
schedule: 'Callable[[float], float]'


In [11]:
oven = HoodedCookingAppliance.create(
    name='oven',
    P_rated=Q_(16, 'kW'),
    F_U=0.16,
    F_rad=0.22,
    schedule=eqp_schedule
)

### Office Appliance

In [12]:
print_doc_string(OfficeAppliance.create)

Creates an `OfficeAppliance` object.

Parameters
----------
name:
    Identifies the office appliance.
P_peak:
    Peak heat gain.
    - Computers: see ASHRAE Fundamentals 2017, Chapter 18, Tables 8A.
      Approximately 90% convective heat gain and 10% radiative heat
      gain.
    - Laptops and laptop docking station: see ASHRAE Fundamentals 2017,
      Chapter 18, Tables 8B. Approximately 75% convective heat gain and
      25% radiative heat gain.
    - Tablet PC: see ASHRAE Fundamentals 2017, Chapter 18, Tables 8C.
    - Monitors: see ASHRAE Fundamentals 2017, Chapter 18, Table 8D.
      Approximately 60% convective heat gain and 40% radiative heat
      gain.
    - Printers and copiers: see ASHRAE Fundamentals 2017, Chapter 18,
      Table 9. Approximately 70% convective heat gain and 30% radiative
      heat gain.
    - Miscellaneous office equipment: see ASHRAE Fundamentals 2017,
      Chapter 18, Table 10.
F_rad:
    The radiative fraction is the radiative part of the office a

In [13]:
computer = OfficeAppliance.create(
    name='computer',
    P_peak=Q_(137, 'W'),
    F_rad=0.1,
    schedule=eqp_schedule
)

### Office Equipment

In [14]:
print_doc_string(OfficeEquipment.create)

Creates an `OfficeEquipment` object.

Parameters
----------
name:
    Identifies the object.
heat_gain_density:
    Heat gain per unit area, aka load factor. See ASHRAE Fundamentals
    2017, Chapter 18, Table 11. The medium heat gain density is likely
    to be appropriate for most standard offices.
floor_area:
    The floor area of the space.
schedule:
    Function with signature `f(t_sol_sec: float) -> float` that takes the
    solar time `t_sol_sec` in seconds from midnight (0 s) and returns
    a float between 0 and 1 which indicates the diversity factor.
F_rad:
    The radiative fraction is the radiative part of the office equipment
    heat gain that goes to the space.

Parameter Types & Defaults
--------------------------
name: 'str'
heat_gain_density: 'Quantity'
floor_area: 'Quantity'
schedule: 'Callable[[float], float]'
F_rad: 'float' = 0.3


In [15]:
office_equipment = OfficeEquipment.create(
    name='office-equipment',
    heat_gain_density=Q_(7.79, 'W / m ** 2'),
    floor_area=Q_(50, 'm **2'),
    schedule=eqp_schedule,
)

### Generic Appliance

In [16]:
print_doc_string(GenericAppliance.create)

Creates a `GenericAppliance` object of which the heat gain components
are already known.

Parameters
----------
name:
    Identifies the appliance.
Q_dot_sen_rd:
    The radiative component of the sensible heat gain.
Q_dot_sen_cv:
    The convective component of the sensible heat gain.
Q_dot_lat:
    The latent heat gain from the appliance.
schedule:
    Function with signature `f(t_sol_sec: float) -> float` that takes the
    solar time `t_sol_sec` in seconds from midnight (0 s) and returns
    a float between 0 and 1, where 0 stands for completely off and 1
    for running at full power.

Parameter Types & Defaults
--------------------------
name: 'str'
Q_dot_sen_rd: 'Quantity'
Q_dot_sen_cv: 'Quantity'
Q_dot_lat: 'Quantity'
schedule: 'Callable[[float], float]'


In [17]:
generic_appliance = GenericAppliance.create(
    name='generic-appliance',
    Q_dot_sen_rd=Q_(50, 'W'),
    Q_dot_sen_cv=Q_(75, 'W'),
    Q_dot_lat=Q_(0, 'W'),
    schedule=eqp_schedule
)

### Equipment Heat Gain

An object of class `EquipmentHeatGain` can group multiple objects of the `Equipment` types.

In [18]:
print_doc_string(EquipmentHeatGain)

Represents the internal heat gain from equipment in the space.

An `EquipmentHeatGain` object contains one or more `Equipment` objects
(see module equipment.py). The user must create these objects with the
necessary input data so that the heat gain of this equipment can be
calculated.

Parameter Types & Defaults
--------------------------
name: str


In [19]:
equipment_heat_gain = EquipmentHeatGain('equipment-heat-gain')

equipment_heat_gain.add_equipment(machine)
equipment_heat_gain.add_equipment(oven)
equipment_heat_gain.add_equipment(computer)
equipment_heat_gain.add_equipment(office_equipment)
equipment_heat_gain.add_equipment(generic_appliance)

In [20]:
for equipment_ID in equipment_heat_gain.equipment.keys():
    print(equipment_ID)

machine_1
oven
computer
office-equipment
generic-appliance


In [21]:
oven = equipment_heat_gain.get_equipment('oven')
print(oven.P_rated.to('W'))

16000.0 watt


To get the equipment heat gain at a certain time of the day, method `Q_dot(...)` is used. It expects the time moment to be expressed in solar seconds from midnight.

Suppose we want to know the equipment heat gain at 10 a.m. on the "design day" in June.

In [22]:
from datetime import time
from hvac.cooling_load_calc import convert_to_solar_seconds

In [23]:
t_clock = time(10, 0, 0)
t_sol_secs = convert_to_solar_seconds(
    clock_time=t_clock,
    date=ReferenceDates.get_date_for('Jun'),
    L_loc=L_loc,
    tz_loc=tz_loc
)
t_sol_secs

29760.38163

In [24]:
Q_eqp_hg_cv, Q_eqp_hg_rad, Q_eqp_hg_lat = equipment_heat_gain.Q_dot(t_sol_secs)
print(
    f"- sensible equipment heat gain, convective fraction = {Q_eqp_hg_cv.to('W'):~P.1f}",
    f"- sensible equipment heat gain, radiant fraction = {Q_eqp_hg_rad.to('W'):~P.1f}",
    f"- latent equipment heat gain = {Q_eqp_hg_lat.to('W'):~P.1f}",
    sep='\n'
)

- sensible equipment heat gain, convective fraction = 4560.4 W
- sensible equipment heat gain, radiant fraction = 4833.2 W
- latent equipment heat gain = 0.0 W


## Internal Heat Gain from Lighting

Module `lighting.py` in subpackage `cooling_load_calc.internal_heat_gains` defines two classes:
- class `LightingFixture`
- class `SpaceLighting`

In [25]:
from hvac.cooling_load_calc import (
    LightingFixture,
    SpaceLighting
)

Objects of these classes are instantiated by calling method `create(...)` of these classes.

### Lighting Fixture

In [26]:
print_doc_string(LightingFixture)

Represents a single lighting fixture or a group of lighting fixtures in
a room. The light heat gain is calculated based on the lighting fixture's
technical specifications.


In [27]:
print_doc_string(LightingFixture.create)

Creates a `LightingFixture` object.
(see ASHRAE Fundamentals 2017, Chapter 18, 2.2 Lighting).

Parameters
----------
name:
    Identifies the lighting fixture.
P_lamp:
    The nominal wattage of the lamp(s) in the fixture without the power
    consumption of ballasts.
F_allowance:
    The ratio of the lighting fixture's power consumption including
    lamps and ballast to the nominal power consumption of the lamps.
    For incandescent lamps, this factor is 1. For fluorescent and other
    lights, it accounts for power consumed by the ballast as well as the
    ballast's effect on lamp power consumption.
F_use:
    The ratio of the lamp wattage in use to the total installed wattage
    for the conditions under which the cooling load estimate is being
    made. For commercial applications such as stores, the use factor is
    usually 1.0.
F_rad:
    The radiative fraction is the radiative part of the lighting heat
    gain that goes to the room (see ASHRAE Fundamentals, Chapter 18,
    

We define a lighting schedule to indicate the period of the day the lights are turned on and when they are turned off.

In [28]:
def light_schedule(t_sol_sec: float) -> float:
    t_clock, _ = convert_to_clock_time(t_sol_sec, date, L_loc, tz_loc)
    t_hour = t_clock.time().hour
    if (0 <= t_hour < 8) or (18 <= t_hour < 24):
        return 1.0
    else:
        return 0.0
    

In [29]:
lighting_fixture = LightingFixture.create(
    name='lighting-fixture',
    P_lamp=Q_(100, 'W'),
    F_allowance=1.1,
    F_use=1.0,
    F_rad=0.73,
    schedule=light_schedule
)

### Space Lighting

In [30]:
print_doc_string(SpaceLighting)

This class can be used to estimate the lighting gain on a per-unit area 
basis (e.g., when final lighting plans are not available).


In [31]:
print_doc_string(SpaceLighting.create)

Creates a `SpaceLighting` object.
(see ASHRAE Fundamentals 2017, Chapter 18, Tables 2 and 3).

Parameters
----------
name:
    Identifies the space lighting.
power_density:
    The maximum lighting power density, i.e. the lighting heat gain per
    square metre (see ASHRAE Fundamentals 2017, Chapter 18, Table 2).
F_space:
    The space fraction, i.e., the fraction of lighting heat gain that
    goes to the room (the other fraction goes to the plenum) (see ASHRAE
    Fundamentals 2017, Chapter 18, Table 3 and Figure 3).
F_rad:
    The radiative fraction is the radiative part of the lighting heat
    gain that goes to the room (see ASHRAE Fundamentals 2017, Chapter
    18, Table 3 and Figure 3).
schedule:
    Function with signature `f(t_sol_sec: float) -> float` that takes the
    solar time `t_sol_sec` in seconds from midnight (0 s) and returns
    a float between 0 and 1 which indicates the diversity factor.
floor_area
    The floor area of the zone.

Parameter Types & Defaults
------

In [32]:
space_lighting = SpaceLighting.create(
    name='space-lighting',
    power_density=Q_(10, 'W / m**2'),
    F_space=0.64,
    F_rad=0.47,
    schedule=light_schedule,
    floor_area=Q_(50, 'm**2')
)

### Lighting Heat Gain

An object of class `LightingHeatGain` can group multiple objects of the `Lighting` classes.

In [33]:
print_doc_string(LightingHeatGain)

Represents the internal heat gain from space lighting.

A `LightingHeatGain` object contains one or more `Lighting` objects
(see module lighting.py). The user must create these objects with the
necessary input data so that the heat gain of the lighting can be
calculated.

Parameter Types & Defaults
--------------------------
name: str


In [34]:
lighting_heat_gain = LightingHeatGain('lighting-heat-gain')

lighting_heat_gain.add_lighting(lighting_fixture)
lighting_heat_gain.add_lighting(space_lighting)

In [35]:
for lighting_ID in lighting_heat_gain.lighting.keys():
    print(lighting_ID)

lighting-fixture
space-lighting


In [36]:
lighting_fixture = lighting_heat_gain.lighting['lighting-fixture']
print(lighting_fixture.P_lamp.to('W'))

100 watt


To get the lighting heat gain at a certain time of the day, method `Q_dot(...)` is used. It expects the time moment to be expressed in solar seconds from midnight. It returns the convective fraction of the sensible lighting heat gain, the radiant fraction, and the latent lighting heat gain (which is however always zero in the case of lighting). 

Suppose we want to know the lighting heat gain at 6 a.m. on the "design day" in June.

In [37]:
t_clock = time(6, 0, 0)
t_sol_secs = convert_to_solar_seconds(
    clock_time=t_clock,
    date=ReferenceDates.get_date_for('Jun'),
    L_loc=L_loc,
    tz_loc=tz_loc
)
t_sol_secs

15360.381630000002

In [38]:
Q_light_hg_cv, Q_light_hg_rad, _ = lighting_heat_gain.Q_dot(t_sol_secs)
print(
    f"- lighting heat gain, convective fraction = {Q_light_hg_cv.to('W'):~P.1f}",
    f"- lighting heat gain, radiant fraction = {Q_light_hg_rad.to('W'):~P.1f}",
    sep='\n'
)

- lighting heat gain, convective fraction = 119.0 W
- lighting heat gain, radiant fraction = 230.7 W


## Internal Heat Gain from People

In [40]:
print_doc_string(PeopleHeatGain.create)

Creates a `PeopleHeatGain` object.
(see ASHRAE Fundamentals 2017, Chapter 18, table 1).

Parameters
----------
name:
    Identifier for the internal heat gain
Q_dot_sen_person :
    Sensible heat release per person.
Q_dot_lat_person :
    Latent heat release per person.
F_rad :
    Fraction of sensible heat release that is radiant.
schedule :
    Function with signature `f(t_sol_sec: float) -> int` that takes the
    solar time in seconds from midnight (0 s) and returns the number
    of people in the thermal zone.

Parameter Types & Defaults
--------------------------
name: str
Q_dot_sen_person: pint.Quantity
Q_dot_lat_person: pint.Quantity
F_rad: float
schedule: Callable[[float], int]


In [41]:
def occupancy_schedule(t_sol_sec: float) -> int:
    t_clock, _ = convert_to_clock_time(t_sol_sec, date, L_loc, tz_loc)
    t_hour = t_clock.time().hour
    if (6 <= t_hour < 12) or (14 <= t_hour < 18):
        return 5
    else:
        return 0
        

In [42]:
people_heat_gain = PeopleHeatGain.create(
    name='people-heat-gain',
    Q_dot_sen_person=Q_(75, 'W'),
    Q_dot_lat_person=Q_(55, 'W'),
    F_rad=0.58,
    schedule=occupancy_schedule
)

To get the people heat gain at a certain time of the day, method `Q_dot(...)` is used. It expects the time moment to be expressed in solar seconds from midnight. It returns the convective fraction of the sensible people heat gain, the radiant fraction, and the latent people heat gain.

Suppose we want to know the people heat gain at 16 a.m. on the "design day" in June.

In [44]:
t_clock = time(16, 0, 0)
t_sol_secs = convert_to_solar_seconds(
    clock_time=t_clock,
    date=ReferenceDates.get_date_for('Jun'),
    L_loc=L_loc,
    tz_loc=tz_loc
)
t_sol_secs

51360.38163

In [45]:
Q_people_hg_cv, Q_people_hg_rad, Q_people_hg_lat = people_heat_gain.Q_dot(t_sol_secs)
print(
    f"- sensible people heat gain, convective fraction = {Q_people_hg_cv.to('W'):~P.1f}",
    f"- sensible people heat gain, radiant fraction = {Q_people_hg_rad.to('W'):~P.1f}",
    f"- latent people heat gain = {Q_people_hg_lat.to('W'):~P.1f}",
    sep='\n'
)

- sensible people heat gain, convective fraction = 157.5 W
- sensible people heat gain, radiant fraction = 217.5 W
- latent people heat gain = 275.0 W
